## **Movie Recommendation on Movielens-1M using PIML Xgboost** 
Compare the prediction with RankFM. Use XGBoost classification, regression and ranker.

#### **Data Loading and Processing**

In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
rating = pd.read_csv("ml-1m/ratings.dat", names=["user_id", "item_id", "rating", "timestamp"], delimiter="::", engine="python")
rating.columns = ['user_id','item_id','rating','timestamp']
rating

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
user_features = pd.read_csv("ml-1m/users.dat", names=["user_id", "gender", "age", "occupation", "zipcode"], delimiter="::", engine="python")
user_features = user_features.drop('zipcode', axis=1)
user_features.occupation = user_features.occupation.astype('str')
user_features.age = user_features.age.astype('str')
user_features = pd.get_dummies(user_features)

user_features.head()

,user_id,gender_F,gender_M,age_1,age_18,age_25,age_35,age_45,age_50,age_56,...,occupation_19,occupation_2,occupation_20,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9
0,1,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,3,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,5,False,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [4]:
import collections

movie_desc = pd.read_csv("ml-1m/movies.dat", names=["item_id", "title", "genres"], delimiter="::", engine="python", encoding="ISO-8859-1")

# Convert genres to lowercase
movie_desc.genres = movie_desc.genres.str.lower()

split_series = movie_desc.genres.str.split('|').apply(lambda x: x)
split_series_dict = split_series.apply(collections.Counter)

multi_hot = pd.DataFrame.from_records(split_series_dict).fillna(value=0).astype('int')

item_features = pd.concat([movie_desc.item_id, multi_hot], axis=1)
item_features

,item_id,animation,children's,comedy,adventure,fantasy,romance,drama,action,crime,thriller,horror,sci-fi,documentary,war,musical,mystery,film-noir,western
0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,3949,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3880,3950,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3881,3951,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
item_names = movie_desc
item_names.head()

,item_id,title,genres
0,1,Toy Story (1995),animation|children's|comedy
1,2,Jumanji (1995),adventure|children's|fantasy
2,3,Grumpier Old Men (1995),comedy|romance
3,4,Waiting to Exhale (1995),comedy|drama
4,5,Father of the Bride Part II (1995),comedy


In [6]:
unique_users = rating.user_id.nunique()
unique_items = rating.item_id.nunique()

print(f"interactions shape {rating.shape}")
print(f"interaction unique users {unique_users}")
print(f"interaction unique items {unique_items}")


interactions shape (1000209, 4)
interaction unique users 6040
interaction unique items 3706


In [7]:
sparsity = 1 - (len(rating)/(unique_users * unique_items))
print(f"interaction matrix sparsity: {round(100 * sparsity, 1)}%")

interaction matrix sparsity: 95.5%


In [8]:
rating.drop(['timestamp'], axis=1, inplace=True)

# set all positive interactions to 1
df_classification = rating.copy()
df_classification['interaction'] = 1
df_classification.drop(['rating'], axis=1, inplace=True)

#### **Classification Approach**
in RankFM, all dataset is positive interaction. To make it as classification prediction, balanced synthesis negative interaction added

In [9]:
import random

all_users = df_classification.user_id.unique()
all_items = df_classification.item_id.unique()

negative_instances = []

for user in all_users:
    user_interacted_item = df_classification[df_classification.user_id == user]['item_id'].unique()
    non_interacted_items = set(all_items) - set(user_interacted_item)
    for item in non_interacted_items:
        negative_instances.append([user, item, 0])

num_negatives = len(rating[df_classification['interaction'] == 1])
sampled_negatives = random.sample(negative_instances, num_negatives)

df_negatives = pd.DataFrame(sampled_negatives, columns=['user_id', 'item_id', 'interaction'])
df_negatives

columns = ['user_id', 'item_id', 'interaction']
balanced_df = pd.concat([df_classification[columns], df_negatives[columns]]).reset_index(drop=True)

In [10]:
df = pd.merge(balanced_df, user_features, on='user_id', how='left')
df = pd.merge(df, item_features, on='item_id', how='left')

#### Data Preparation with PIML

In [11]:
from piml import Experiment
from piml.models import XGB2Regressor, XGB2Classifier
from sklearn.model_selection import train_test_split

user_item = df[['user_id', 'item_id']]

# Prepare the features and labels
X = df.drop(['user_id', 'item_id', 'interaction'], axis=1)
y = df['interaction']

# Perform train-test split
train_x, test_x, train_y, test_y, user_item_train, user_item_test = train_test_split(
    X, y, user_item, test_size=0.25, random_state=42
)

# Get the indices for train and test sets
train_indices = np.array(train_x.index)
test_indices = np.array(test_x.index)

exp = Experiment(highcode_only=True)
exp.data_loader(data=df, silent=True)
exp.data_summary(feature_exclude=['user_id', 'item_id'], silent=True)
exp.data_prepare(target='interaction', task_type='classification', train_idx=train_indices, test_idx=test_indices, silent=True)
# exp.feature_select(threshold=0.98, method="pfi", figsize=(6, 5))


In [12]:
print("train shape: {}".format(train_x.shape))
print("valid shape: {}".format(test_x.shape))

train_users = np.sort(user_item_train.user_id.unique())
valid_users = np.sort(user_item_test.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(user_item_train.item_id.unique())
valid_items = np.sort(user_item_test.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

train shape: (1500313, 48)
valid shape: (500105, 48)
train users: 6040
valid users: 6040
cold-start users: set()
train items: 3706
valid items: 3706
cold-start items: set()


#### Model Training

In [13]:
exp.model_train(model=XGB2Classifier(), name="XGB2")
exp.model_diagnose(model="XGB2", show="accuracy_table")

,ACC,AUC,F1,LogLoss,Brier
,,,,,
Train,0.6293,0.6822,0.6116,0.6396,0.2245
Test,0.6287,0.6822,0.6115,0.6395,0.2245
Gap,-0.0006,0.0000,-0.0001,-0.0001,-0.0000


In [14]:
# exp.model_interpret(model="XGB2", show="global_ei", figsize=(5, 4))


In [15]:
# exp.model_interpret(model="XGB2", show="global_fi", figsize=(5, 4))

In [16]:
# exp.model_interpret(model="XGB2", show="local_ei", sample_id=0, original_scale=True, figsize=(5, 4))

In [17]:
# exp.model_interpret(model="XGB2", show="local_fi", sample_id=0, original_scale=True, figsize=(5, 4))


#### Use RankFM Validation Data

In [18]:
np.random.seed(42)

interactions = rating.copy()
interactions['random'] = np.random.random(size=len(interactions))
test_pct = 0.25

train_mask = interactions['random'] <  (1 - test_pct)
valid_mask = interactions['random'] >= (1 - test_pct)

interactions_train = interactions[train_mask][['user_id', 'item_id']]
interactions_valid = interactions[valid_mask][['user_id', 'item_id']]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.item_id.unique())
valid_items = np.sort(interactions_valid.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))


train shape: (750042, 2)
valid shape: (250167, 2)
train users: 6040
valid users: 6040
cold-start users: set()
train items: 3670
valid items: 3507
cold-start items: {3842, 2308, 2438, 3220, 3607, 2584, 1820, 2845, 2591, 545, 1316, 2214, 1832, 1579, 3376, 1714, 1843, 2226, 2742, 311, 826, 2235, 3517, 1470, 576, 2895, 601, 3291, 989, 1630, 2909, 868, 2277, 2039, 3065, 2556}


#### Predict on test data  
Make the top-10 K 

In [19]:
k = 10

def pivot_table_recommendation(recommendation):
    # Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
    pivot_recommendations = recommendation.pivot(index='user_id', columns='rank', values='item_id').reset_index()

    # Set user_id as the index
    pivot_recommendations.set_index('user_id', inplace=True)

    # Remove the 'rank' column
    pivot_recommendations.columns.name = None
    pivot_recommendations.columns = [f'{int(rank)}' for rank in pivot_recommendations.columns]
    pivot_recommendations.index.name = None

    return pivot_recommendations

#### Predict on Test Dataset

In [20]:
df_test = pd.merge(user_item_test, user_features, on='user_id', how='left')
df_test = pd.merge(df_test, item_features, on='item_id', how='left')

dpredict = df_test.drop(['user_id', 'item_id'], axis=1)

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_interaction'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_interaction'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,1375,588,745,3114,3745,2692,266,3479,1,2848
2,1544,21,1917,1253,356,1378,185,2812,292,2427
3,3612,3893,1425,653,1772,1358,3534,145,1259,2470
4,368,1683,1954,2556,2764,3666,3151,1036,1214,247
5,1565,1092,1171,2774,2585,593,2427,1057,1391,2289


**Notes:** 
- RankFM able to generate movie_id based on user_id input, while XGBoost is predict the interaction given user_id and movie_id

#### Predict on RankFM Validation dataset

In [21]:
df_test = pd.merge(interactions_valid, user_features, on='user_id', how='left')
df_test = pd.merge(df_test, item_features, on='item_id', how='left')

dpredict = df_test.drop(['user_id', 'item_id'], axis=1)

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_interaction'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_interaction'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,661.0,48.0,588.0,783.0,2692.0,3114.0,2804.0,938.0,2398.0,1907.0
2,2916.0,2881.0,3030.0,434.0,292.0,3257.0,2278.0,2028.0,1408.0,480.0
3,3421.0,1641.0,3534.0,3868.0,1079.0,653.0,2167.0,1580.0,3114.0,3552.0
4,480.0,1196.0,1198.0,3418.0,2366.0,1387.0,3527.0,2947.0,NaN,NaN
5,1175.0,1392.0,860.0,215.0,3578.0,3793.0,1610.0,2058.0,1909.0,229.0


**Notes:** 
- The results is looks totally differences than RankFM and few is NaN

In [22]:
k = 10

test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

def hit_rate():
    hit_rate = np.mean([int(len(set(pivot_recommendations.loc[u]) & test_users_items[u]) > 0) for u in comm_user])
    return hit_rate

def reciprocal_rank():
    match_indexes = [np.where(pivot_recommendations.loc[u].isin(set(pivot_recommendations.loc[u]) & test_users_items[u]))[0] for u in comm_user]
    reciprocal_rank = np.mean([1 / (np.min(index) + 1) if len(index) > 0 else 0 for index in match_indexes])

    return reciprocal_rank

def dcg():
    match_indexes = [np.where(pivot_recommendations.loc[u].isin(set(pivot_recommendations.loc[u]) & test_users_items[u]))[0] for u in comm_user]
    discounted_cumulative_gain = np.mean([np.sum(1 / np.log2(index + 2)) if len(index) > 0 else 0 for index in match_indexes])
    
    return discounted_cumulative_gain

def precision():
    precision = np.mean([len(set(pivot_recommendations.loc[u]) & test_users_items[u]) / len(pivot_recommendations.loc[u]) for u in comm_user])

    return precision

def recall():
    recall = np.mean([len(set(pivot_recommendations.loc[u]) & test_users_items[u]) / len(test_users_items[u]) for u in comm_user])

    return recall

print("EVALUATION CLASSIFICATION TESTSET ONLY\n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION CLASSIFICATION TESTSET ONLY

hit_rate: 1.000
reciprocal_rank: 1.000
dcg: 4.315
precision: 0.928
recall: 0.510


**Notes:** 

- XGB2 outperform because its leverage rich data from user and item features
- Next, we test on unseen data to check whether its over-fitting

#### Prediction on All Combination Data (excluded training data)

In [23]:
# Assuming all_users is a list of all user IDs and all_items is a list of all item IDs
# Create a DataFrame for all user-item pairs
all_user_item_pairs = pd.DataFrame([(user, item) for user in all_users for item in all_items], columns=['user_id', 'item_id'])

# Remove training data from all_user_item_pairs
all_user_item_pairs = all_user_item_pairs[~all_user_item_pairs.isin(user_item_train[['user_id', 'item_id']])].dropna()


In [24]:
# Add user and item features
all_pairs_completed = pd.merge(all_user_item_pairs, user_features, on='user_id', how='left')
all_pairs_completed = pd.merge(all_pairs_completed, item_features, on='item_id', how='left')

dpredict = all_pairs_completed.drop(['user_id', 'item_id'], axis=1)

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
all_pairs_completed['predicted_interaction'] = xgb2.predict(dpredict)

In [25]:
# Rank the predictions
all_pairs_completed['rank'] = all_pairs_completed.groupby('user_id')['predicted_interaction'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = all_pairs_completed[all_pairs_completed['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1.0,745.0,588.0,1.0,2692.0,3114.0,2126.0,3035.0,1544.0,3257.0,110.0
2.0,661.0,2355.0,1197.0,1287.0,594.0,595.0,2797.0,1270.0,527.0,48.0
3.0,661.0,2355.0,1197.0,1287.0,2804.0,594.0,919.0,595.0,2918.0,2791.0
4.0,661.0,2355.0,1197.0,1287.0,594.0,919.0,595.0,2797.0,1270.0,527.0
5.0,661.0,914.0,2355.0,1197.0,1287.0,2804.0,594.0,919.0,595.0,2918.0


#### Evaluation Unseen Data Prediction Results

In [26]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION CLASSIFICATION UNSEEN DATA EXCLUDED TRAINING\n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION CLASSIFICATION UNSEEN DATA EXCLUDED TRAINING

hit_rate: 0.383
reciprocal_rank: 0.114
dcg: 0.242
precision: 0.057
recall: 0.017


**Notes:** 

- Its significant lower when predicting over unseen data and back testing into test data

#### **Regression Approach**

In [27]:
df = pd.merge(rating, user_features, on='user_id', how='left')
df = pd.merge(df, item_features, on='item_id', how='left')

user_item = df[['user_id', 'item_id']]

# Prepare the features and labels
X = df.drop(['user_id', 'item_id', 'rating'], axis=1)
y = df['rating']

# Perform train-test split
train_x, test_x, train_y, test_y, user_item_train, user_item_test = train_test_split(
    X, y, user_item, test_size=0.25, random_state=42
)

# Get the indices for train and test sets
train_indices = np.array(train_x.index)
test_indices = np.array(test_x.index)

exp = Experiment(highcode_only=True)
exp.data_loader(data=df, silent=True)
exp.data_summary(feature_exclude=['user_id', 'item_id', 'rating'], silent=True)
exp.data_prepare(target='rating', task_type='regression', train_idx=train_indices, test_idx=test_indices, silent=True)
# exp.feature_select(threshold=0.98, method="pfi", figsize=(6, 5))

In [28]:
exp.model_train(model=XGB2Regressor(), name="XGB2")
exp.model_diagnose(model="XGB2", show="accuracy_table")

,MSE,MAE,R2
,,,
Train,0.0736,0.2230,0.0548
Test,0.0740,0.2235,0.0543
Gap,0.0003,0.0005,-0.0004


In [29]:
df_test = pd.merge(user_item_test, user_features, on='user_id', how='left')
df_test = pd.merge(df_test, item_features, on='item_id', how='left')

dpredict = df_test.drop(['user_id', 'item_id'], axis=1)

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_rating'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_rating'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,608.0,588.0,3186.0,1193.0,3105.0,2398.0,1836.0,1246.0,595.0,2804.0
2,1196.0,1090.0,2067.0,110.0,2427.0,1945.0,1084.0,1953.0,1957.0,3468.0
3,1196.0,2081.0,1968.0,3168.0,3671.0,1259.0,1394.0,653.0,1270.0,2470.0
4,1210.0,260.0,1201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1617.0,913.0,41.0,1192.0,1191.0,2323.0,162.0,2427.0,919.0,3513.0


#### Test with RankFM Validation Dataset

In [30]:
df_test = pd.merge(interactions_valid, user_features, on='user_id', how='left')
df_test = pd.merge(df_test, item_features, on='item_id', how='left')

dpredict = df_test.drop(['user_id', 'item_id'], axis=1)

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
df_test['predicted_rating'] = xgb2.predict(dpredict)

# Rank the predictions
df_test['rank'] = df_test.groupby('user_id')['predicted_rating'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test[df_test['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1,608.0,588.0,2398.0,1246.0,661.0,783.0,48.0,2804.0,938.0,3114.0
2,1196.0,920.0,2028.0,3030.0,590.0,1213.0,1084.0,1953.0,1293.0,1873.0
3,1968.0,3671.0,3114.0,3421.0,1641.0,3534.0,3868.0,1079.0,3552.0,653.0
4,1196.0,3418.0,1198.0,3527.0,480.0,2366.0,2947.0,1387.0,NaN,NaN
5,913.0,1191.0,2427.0,229.0,3513.0,908.0,3409.0,1759.0,501.0,2070.0


In [31]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION REGRESSION TEST SET ONLY \n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION REGRESSION TEST SET ONLY 

hit_rate: 1.000
reciprocal_rank: 1.000
dcg: 4.315
precision: 0.928
recall: 0.510


In [32]:
# Assuming all_users is a list of all user IDs and all_items is a list of all item IDs
# Create a DataFrame for all user-item pairs
all_user_item_pairs = pd.DataFrame([(user, item) for user in rating.user_id.unique() for item in rating.item_id.unique()], columns=['user_id', 'item_id'])

# Remove training data from all_user_item_pairs
all_user_item_pairs = all_user_item_pairs[~all_user_item_pairs.isin(user_item_train[['user_id', 'item_id']])].dropna()

all_pairs_completed = pd.merge(all_user_item_pairs, user_features, on='user_id', how='left')
all_pairs_completed = pd.merge(all_pairs_completed, item_features, on='item_id', how='left')

In [33]:
dpredict = all_pairs_completed.drop(['user_id', 'item_id'], axis=1)

xgb2 = exp.get_model("XGB2")

# Predict the interaction probabilities
all_user_item_pairs['predicted_rating'] = xgb2.predict(dpredict)

In [34]:
# Rank the predictions
all_user_item_pairs['rank'] = all_user_item_pairs.groupby('user_id')['predicted_rating'].rank(method='first', ascending=False)

# Filter to get top 10 predictions for each user
top_10_recommendations = all_user_item_pairs[all_user_item_pairs['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
1.0,1298.0,707.0,1248.0,1152.0,1260.0,1151.0,3435.0,2726.0,3364.0,1068.0
2.0,707.0,1248.0,1152.0,1260.0,1298.0,1196.0,745.0,1151.0,3435.0,2726.0
3.0,707.0,1248.0,1152.0,1260.0,745.0,1151.0,1298.0,3435.0,2726.0,3364.0
4.0,707.0,1248.0,1152.0,1260.0,1298.0,745.0,1151.0,3435.0,2726.0,3364.0
5.0,707.0,1248.0,1152.0,1260.0,1298.0,745.0,1151.0,3435.0,2726.0,3364.0


In [35]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION REGRESSION UNSEEN DATA EXCLUDED TRAINING \n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION REGRESSION UNSEEN DATA EXCLUDED TRAINING 

hit_rate: 0.113
reciprocal_rank: 0.035
dcg: 0.060
precision: 0.013
recall: 0.004


#### **XGBRanker Approach**
We are using the classification data that have positive and negative balance

In [36]:
df = pd.merge(balanced_df, user_features, on='user_id', how='left')
df = pd.merge(df, item_features, on='item_id', how='left')

In [37]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
train_idx, test_indices = next(gss.split(df, groups=df.user_id))

train_data = df.iloc[train_idx]
test_data = df.iloc[test_indices]

X_train = train_data.drop(columns=['interaction'])
y_train = train_data['interaction']
X_test = test_data.drop(columns=['interaction'])
y_test = test_data['interaction']


In [38]:
train_groups = X_train.groupby('user_id').size().to_numpy()
test_groups = X_test.groupby('user_id').size().to_numpy()

In [39]:
import xgboost as xgb 

dtrain = xgb.DMatrix(X_train, label=y_train)
dtrain.set_group(train_groups)

dtest = xgb.DMatrix(X_test, label=y_test)
dtest.set_group(test_groups)

params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'learning_rate': 0.1,
    'max_depth': 6,
    'verbose': 0 
}

bst = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtest, 'test')], early_stopping_rounds=10)

[0]	test-ndcg:1.00000
[1]	test-ndcg:1.00000
[2]	test-ndcg:1.00000
[3]	test-ndcg:1.00000
[4]	test-ndcg:1.00000
[5]	test-ndcg:1.00000
[6]	test-ndcg:1.00000
[7]	test-ndcg:1.00000
[8]	test-ndcg:1.00000
[9]	test-ndcg:1.00000


In [40]:
test_data['predicted_score'] = bst.predict(dtest)
test_data['rank'] = test_data.groupby('user_id')['predicted_score'].rank(method='first', ascending=False)

top_10_recommendations = test_data[test_data['rank'] <= 10]

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = pivot_table_recommendation(top_10_recommendations)
pivot_recommendations.head()

,1,2,3,4,5,6,7,8,9,10
9,2268,1466,1393,861,1682,3717,508,3793,720,367
13,2987,648,2628,2054,1259,589,1690,2,153,1331
15,3421,648,3354,2485,141,2126,2058,3798,2997,653
16,2987,2555,2629,1682,2485,2701,2568,3004,1269,2713
18,2987,2989,2622,648,2628,1682,1683,1688,2123,2052


In [41]:
test_users_items = df_test.groupby('user_id')['item_id'].apply(set).to_dict()
test_users = list(test_users_items.keys())
comm_user = pivot_recommendations.index.values

print("EVALUATION XGB RANKER ON TESTSET\n")

print("hit_rate: {:.3f}".format(hit_rate()))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank()))
print("dcg: {:.3f}".format(dcg()))
print("precision: {:.3f}".format(precision()))
print("recall: {:.3f}".format(recall()))

EVALUATION XGB RANKER ON TESTSET

hit_rate: 0.943
reciprocal_rank: 0.465
dcg: 1.137
precision: 0.248
recall: 0.157


#### **Result Comparison**

| Metrics | Classification | Regressor | Rank Based | RankFM |
| --- | --- | --- | --- | --- | 
| hit_rate | 0.383 | 0.113 | 0.943 | 0.788 |
| reciprocal_rank | 0.114 | 0.035 | 0.465 | 0.334 |
| dcg | 0.242 | 0.060 | 1.137 | 0.718 |
| precision | 0.057 | 0.013 | 0.248 | 0.156 |
| recall | 0.017 | 0.004 | 0.157 | 0.072 |
